In [1]:
using Pkg
Pkg.add(PackageSpec(name="JWAS", rev="master"))
Pkg.update()
Pkg.add("DataFrames")
Pkg.add("CSV")
using JWAS,JWAS.Datasets,DataFrames,CSV,Statistics

    Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
   Resolving package versions...
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`
    Updating registry at `C:\Users\aisha\.julia\registries\General.toml`
    Updating git-repo `https://github.com/reworkhow/JWAS.jl.git`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`
   Resolving package versions...
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\aisha\.julia\environments\v1.10\Manifest.toml`


In [2]:
phenofile = dataset("C:/Users/aisha/Desktop/Grad_School/ROTATIONS/Jack_Lab/Stress_project/Analyses/SWIM_imputation/QTLmarkers/Hormones.csv")
genofile = dataset("C:/Users/aisha/Desktop/Grad_School/ROTATIONS/Jack_Lab/Stress_project/Analyses/GWAS/dummy_genotypes.txt")
phenotypes = CSV.read(phenofile,DataFrame,delim = ',',header=true,missingstrings=["NA"])
phenotypes = dropmissing(phenotypes, :qNurlogCort)
first(phenotypes,5)

┌ Warning: `missingstrings` keyword argument is deprecated; pass a `Vector{String}` to `missingstring` instead
└ @ CSV C:\Users\aisha\.julia\packages\CSV\aoJqo\src\context.jl:344


Row,ID,Batch,Company,SowID,EntryAge,Cortisol,Hair1_GrindingDate,C_ExtractionDate,C_GrindingToExtractionTime,qNurlogCort,SW_144836353,SW_144841591,SW_144792892,SW_144876147,SW_144827094,SW_144841166,qNurPenBatch,SW_144783524,SW_144783527,cNurlogCort,cNurPenBatch,GrindingDate2,cNur_groundstorage
,Int64,Int64?,String1?,String15?,Int64?,Float64?,String15?,String15?,Int64?,Float64,Int64,Int64,Int64,Int64,Int64,Int64,String7?,Int64,Int64,Float64?,String7?,String15?,Int64?
1,4935,81,E,E8B260,22,16.499,4/20/2022,5/5/2022,15,2.8033,0,0,0,0,0,0,81_1,0,0,2.14908,81_D02,29/08/2022,85
2,4936,81,E,W3W953,22,13.6,4/20/2022,5/5/2022,15,2.61007,0,0,0,0,0,0,81_1,0,0,1.757,81_B04,29/08/2022,85
3,4937,81,E,E8B205,17,23.1791,4/20/2022,5/5/2022,15,3.14325,0,0,0,0,0,0,81_1,0,0,2.40153,81_D08,29/08/2022,85
4,4938,81,E,E8B411,19,16.9478,6/9/2022,5/5/2022,missing,2.83014,1,0,1,0,1,1,81_1,1,1,-0.71744,81_B04,30/08/2022,84
5,4939,81,E,3240,16,13.7074,4/22/2022,6/13/2022,52,2.61794,0,0,0,0,0,0,81_1,0,0,1.46326,81_B06,29/08/2022,135


In [3]:
MY_G3=[0.263606 0
    0 0.555353]
myPi=Dict([1.0; 1.0]=>0.000001,[1.0; 0.0]=>0.000999,[0.0; 1.0]=>0.000999,[0.0; 0.0]=>0.998001)
genotypes=get_genotypes(genofile,header=true,separator=',',MY_G3,method="BayesB",quality_control=false, Pi = myPi, estimatePi = false,estimate_scale=false);


The delimiterd in dummy_genotypes.txt is ','. The header (marker IDs) is provided in dummy_genotypes.txt.
Genotype informatin:
#markers: 4999; #individuals: 4095


In [4]:
# Step 3: Build Model Equations
MY_model_equations = "qNurlogCort = Batch + EntryAge  + C_GrindingToExtractionTime + SowID + qNurPenBatch + C_ExtractionDate + Hair1_GrindingDate + genotypes
             Cortisol = Batch + EntryAge  + C_GrindingToExtractionTime + SowID + qNurPenBatch + C_ExtractionDate + Hair1_GrindingDate + genotypes";
MY_R= [0.567401 0.568697
    0.568697 3.75174]
MY_model=build_model(MY_model_equations,MY_R);#

In [5]:
# Step 4: Set Factors or Covariates
set_covariate(MY_model, "EntryAge");
set_covariate(MY_model, "C_GrindingToExtractionTime");

set_random(MY_model,"qNurPenBatch"); 
set_random(MY_model,"SowID"); 
set_random(MY_model,"C_ExtractionDate"); 
set_random(MY_model,"Hair1_GrindingDate"); 

In [6]:
out=runMCMC(MY_model,phenotypes,missing_phenotypes=true,estimate_variance=true,chain_length=100,burnin=0,output_heritability=true,output_samples_frequency=10);

The folder results is created to save results.
Checking genotypes...
Checking phenotypes...
Individual IDs (strings) are provided in the first column of the phenotypic data.
Predicted values for individuals of interest will be obtained as the summation of Any[] (Note that genomic data is always included for now).Missing values are found in column Batch for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. Users may impute missing values before the analysis 
Missing values are found in column EntryAge for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. Users may impute missing values before the analysis 
Missing values are found in column C_GrindingToExtractionTime for some observations.Effects of this variable on such observations are considered as zeros. It will be used in the estimation of this effect. User

running MCMC ...   2%|█                                  |  ETA: 0:05:18

The file results/MCMC_samples_heritability.txt is created to save MCMC samples for heritability.


running MCMC ... 100%|███████████████████████████████████| Time: 0:00:14




The version of Julia and Platform in use:

Julia Version 1.10.0
Commit 3120989f39 (2023-12-25 18:01 UTC)
Build Info:
  Official https://julialang.org/ release
Platform Info:
  OS: Windows (x86_64-w64-mingw32)
  CPU: 16 × AMD Ryzen 7 5700U with Radeon Graphics         
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-15.0.7 (ORCJIT, znver2)
  Threads: 2 on 16 virtual cores


The analysis has finished. Results are saved in the returned variable and text files. MCMC samples are saved in text files.




In [7]:
cd("C:/Users/aisha/results")
out=GWAS(MY_model,"dummy_mapfile.txt","MCMC_samples_marker_effects_genotypes_Cortisol.txt","MCMC_samples_marker_effects_genotypes_qNurlogCort.txt",header=true,window_size="0.25 Mb",sliding_window=false,GWAS=true,genetic_correlation=true)
#sort(out, :estimate_cor, rev=true)
#CSV.write("winVarPropsBivar.txt",out)
pwd()

Compute the posterior probability of association of the genomic window that explains more than 0.001 of the total genetic variance.

running GWAS...  20%|████████                            |  ETA: 0:00:02

running GWAS... 100%|████████████████████████████████████| Time: 0:00:01
running GWAS... 100%|████████████████████████████████████| Time: 0:00:00
calculating genomic correlation... 100%|█████████████████| Time: 0:00:00


"C:\\Users\\aisha\\results"

In [8]:
?runMCMC

search: runMCMC



```
runMCMC(model::MME,df::DataFrame;
        #Data
        heterogeneous_residuals           = false,
        #MCMC
        chain_length::Integer             = 100,
        starting_value                    = false,
        burnin::Integer                   = 0,
        output_samples_frequency::Integer = chain_length>1000 ? div(chain_length,1000) : 1,
        update_priors_frequency::Integer  = 0,
        #Methods
        single_step_analysis            = false, #parameters for single-step analysis
        pedigree                        = false, #parameters for single-step analysis
        fitting_J_vector                = true,  #parameters for single-step analysis
        categorical_trait               = false,
        censored_trait                  = false,
        causal_structure                = false,
        mega_trait                      = false,
        missing_phenotypes              = true,
        constraint                      = false,
        #Genomic Prediction
        outputEBV                       = true,
        output_heritability             = true,
        prediction_equation             = false,
        #MISC
        seed                            = false,
        printout_model_info             = true,
        printout_frequency              = chain_length+1,
        big_memory                      = false,
        double_precision                = false,
        ##MCMC samples (defaut to marker effects and hyperparametes (variance components))
        output_folder                     = "results",
        output_samples_for_all_parameters = false,
        ##for deprecated JWAS
        methods                         = "conventional (no markers)",
        Pi                              = 0.0,
        estimatePi                      = false)
```

**Run MCMC for Bayesian Linear Mixed Models with or without estimation of variance components.**

  * Markov chain Monte Carlo

      * The first `burnin` iterations are discarded at the beginning of a MCMC chain of length `chain_length`.
      * Save MCMC samples every `output_samples_frequency` iterations, defaulting to `chain_length/1000`, to a folder `output_folder`, defaulting to `results`. MCMC samples for hyperparametes (variance componets) and marker effects are saved by default. MCMC samples for location parametes can be saved using function `output_MCMC_samples()`. Note that saving MCMC samples too frequently slows down the computation.
      * The `starting_value` can be provided as a vector for all location parameteres and marker effects, defaulting to `0.0`s. The order of starting values for location parameters and marker effects should be the order of location parameters in the Mixed Model Equation for all traits (This can be obtained by getNames(model)) and then markers for all traits (all markers for trait 1 then all markers for trait 2...).
      * Miscellaneous Options

          * Priors are updated every `update_priors_frequency` iterations, defaulting to `0`.
  * Methods

      * Single step analysis is allowed if `single_step_analysis` = `true` and `pedigree` is provided.
      * Miscellaneous Options

          * Missing phenotypes are allowed in multi-trait analysis with `missing_phenotypes`=true, defaulting to `true`.
          * Catogorical Traits are allowed if `categorical_trait`=true, defaulting to `false`. Phenotypes should be coded as 1,2,3...
          * Censored traits are allowed if the upper bounds are provided in `censored_trait` as an array, and lower bounds are provided as phenotypes.
          * If `constraint`=true, defaulting to `false`, constrain residual covariances between traits to be zeros.
          * If `causal_structure` is provided, e.g., causal_structure = [0.0 0.0 0.0;1.0 0.0 0.0;1.0 0.0 0.0] for trait 1 -> trait 2 and trait 1 -> trait 3 (column index affacts row index, and a lower triangular matrix is required), phenotypic causal networks will be incorporated using structure equation models.
  * Genomic Prediction

      * Predicted values for individuals of interest can be obtained based on a user-defined prediction equation `prediction_equation`, e.g., "y1:animal + y1:age".

    For now, genomic data is always included. Genetic values including effects defined with genotype and pedigree information are returned if `prediction_equation`= false, defaulting to `false`.

      * Individual estimted genetic values and prediction error variances (PEVs) are returned if `outputEBV`=true, defaulting to `true`. Heritability and genetic

    variances are returned if `output_heritability`=`true`, defaulting to `true`. Note that estimation of heritability is computaionally intensive.
  * Miscellaneous Options

      * Print out the model information in REPL if `printout_model_info`=true; print out the monte carlo mean in REPL with `printout_frequency`, defaulting to `false`.
      * If `seed`, defaulting to `false`, is provided, a reproducible sequence of numbers will be generated for random number generation.
      * If `big_memory`=true, defaulting to `false`, a machine with  lots of memory is assumed which may speed up the analysis.


In [9]:
?GWAS

search: GWAS



```
GWAS(marker_effects_file,model;header=true,window_size=100,threshold=0.001)
```

run genomic window-based GWAS without a map file

  * MCMC samples of marker effects are stored in **marker*effects*file** with delimiter ','.
  * **window_size** is either a constant (identical number of markers in each window) or an array of number of markers in each window
  * **model** is either the model::MME used in analysis or the genotypic covariate matrix M::Array
  * File format:

---

```
GWAS(marker_effects_file;header=true)
```

Compute the model frequency for each marker (the probability the marker is included in the model) using samples of marker effects stored in **marker*effects*file**.

---

```
GWAS(model,map_file,marker_effects_file...;
     window_size = "1 Mb",sliding_window = false,
     GWAS = true, threshold = 0.001,
     genetic_correlation = false,
     header = true)
```

run genomic window-based GWAS

  * MCMC samples of marker effects are stored in **marker*effects*file** with delimiter ','.
  * **model** is either the model::MME used in analysis or the genotype cavariate matrix M::Array
  * **map_file** has the (sorted) marker position information with delimiter ','. If the map file is not provided, i.e., **map_file**=`false`, a fake map file will be generated with **window_size** markers in each 1 Mb window, and each 1 Mb window will be tested.
  * If two **marker*effects*file** are provided, and **genetic_correlation** = true, genomic correlation for each window is calculated.
  * Statistics are computed for nonoverlapping windows of size *window_size* by default. If **sliding_window** = true, those for overlapping sliding windows are calculated.
  * map file format:

```
markerID,chromosome,position
m1,1,16977
m2,1,434311
m3,1,1025513
m4,2,70350
m5,2,101135
```
